In [1]:
import numpy as np
from matplotlib import pyplot
import math
import pandas as pd

In [7]:
L=3         #Length of Nozzle
N=31        #No. of nodes
dx=L/(N-1)
C=0.5       #Courant Number
rho=np.zeros(N)         #density
V=np.zeros(N)           #velocity
T=np.zeros(N)           #temperature
Time=5000               #Timesteps
a=np.zeros(N)           #Non dimensional speed of sound
A=np.zeros(N)           #Cross-section area
M=np.zeros(N)           #Mach Number
P=np.zeros(N)           #Pressure
gamma=1.4               #cp/cv
point=np.zeros(N)
distratio=np.zeros(N)
t=0                     #currenttimestep

In [8]:
#Nozzle shape
for i in range(N):
    if i<16:
        A[i]=1+2.2*(i*dx-1.5)*(i*dx-1.5)
    else:
        A[i]=1+0.2223*(i*dx-1.5)*(i*dx-1.5)
    point[i]=i
    distratio[i]=i*dx/L

In [9]:
#Initial Conditions
for i in range(N):
    rho[i]=1-0.023*i*dx/L
    T[i]=1-0.009333*i*dx/L
    V[i]=0.05+0.11*i*dx/L

In [10]:
dt=np.zeros(N)
a=np.sqrt(T)
#gradients
drho1=np.zeros(N)
drho2=np.zeros(N)
dV1=np.zeros(N)
dV2=np.zeros(N)
dT1=np.zeros(N)
dT2=np.zeros(N)
lnA=np.zeros(N)
#initializing predicted values
prho=np.zeros(N)
pV=np.zeros(N)
pT=np.zeros(N)

In [11]:
#loop
for t in range (Time):
    if t%500==00:
         print(t)
   
    for i in range (N):
        dt[i]=C*dx/(a[i]+V[i])
    DT=min(dt)
    #predictor
    lnA=np.log(A)
    for i in range (0,N-1):
        drho1[i]=-rho[i]*(V[i+1]-V[i])/dx-rho[i]*V[i]*(lnA[i+1]-lnA[i])/dx-V[i]*(rho[i+1]-rho[i])/dx
        dV1[i]=-V[i]*(V[i+1]-V[i])/dx-1/gamma*((T[i+1]-T[i])/dx+T[i]/rho[i]*(rho[i+1]-rho[i])/dx)
        dT1[i]=-V[i]*(T[i+1]-T[i])/dx-(gamma-1)*T[i]*((V[i+1]-V[i])/dx+V[i]*(lnA[i+1]-lnA[i])/dx)
    for i in range (0,N-1):
        prho[i]=rho[i]+drho1[i]*DT
        pV[i]=V[i]+dV1[i]*DT
        pT[i]=T[i]+dT1[i]*DT
    for i in range (1,N):
        #corrector
        drho2[i]=-prho[i]*(pV[i]-pV[i-1])/dx-prho[i]*pV[i]*(lnA[i]-lnA[i-1])/dx-pV[i]*(prho[i]-prho[i-1])/dx
        dV2[i]=-pV[i]*(pV[i]-pV[i-1])/dx-1/gamma*((pT[i]-pT[i-1])/dx+pT[i]/prho[i]*(prho[i]-prho[i-1])/dx)
        dT2[i]=-pV[i]*(pT[i]-pT[i-1])/dx-(gamma-1)*pT[i]*((pV[i]-pV[i-1])/dx+pV[i]*(lnA[i]-lnA[i-1])/dx)
    for i in range (1,N):
        rho[i]=rho[i]+DT*0.5*(drho1[i]+drho2[i])
        V[i]=V[i]+DT*0.5*(dV1[i]+dV2[i])
        T[i]=T[i]+DT*0.5*(dT1[i]+dT2[i])
    
    #boundary conditions
    V[0]=2*V[1]-V[2]
    rho[0]=1
    T[0]=1
    V[N-1]=2*V[N-2]-V[N-3]
    P[N-1]=0.93
    T[N-1]=2*T[N-2]-T[N-3]
    rho[N-1]=P[N-1]/T[N-1]
    a=np.sqrt(T)
    M=V/a
    P=rho*T

0


C:\Users\Tarun\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


500
1000
1500
2000
2500
3000
3500
4000
4500


In [12]:
print("Final Result")
Result={'I':point,'x/L':distratio,'A/ThroatArea':A,'rho/rho0':rho,'V/V0':V,'T/T0':T,'P/P0':P,'M':M}
Resulttable=pd.DataFrame(Result)
Resulttable

Final Result


,I,x/L,A/ThroatArea,rho/rho0,V/V0,T/T0,P/P0,M
0,0.0,0.000000,5.950000,1.000000,0.077876,1.000000,1.000000,0.077876
1,1.0,0.033333,5.312000,0.998133,0.088458,0.999253,0.997387,0.088491
2,2.0,0.066667,4.718000,0.997661,0.099040,0.999064,0.996727,0.099087
3,3.0,0.100000,4.168000,0.996045,0.112478,0.998417,0.994468,0.112567
4,4.0,0.133333,3.662000,0.994351,0.128011,0.997737,0.992100,0.128156
5,5.0,0.166667,3.200000,0.991793,0.146808,0.996711,0.988531,0.147050
6,6.0,0.200000,2.782000,0.988388,0.169262,0.995341,0.983783,0.169658
7,7.0,0.233333,2.408000,0.983594,0.196330,0.993410,0.977112,0.196980
8,8.0,0.266667,2.078000,0.976936,0.228833,0.990716,0.967866,0.229902
9,9.0,0.300000,1.792000,0.967649,0.267672,0.986945,0.955016,0.269437
